## Head direction check


We need to ensure that 0, pi/2, pi, and -pi/2 correspond to the same East,North,West,South direction across our sessions. 

The old version of the tracking system (positrack) did not have the head-direction aligned with heading. We need to correct this.



In [1]:
%load_ext autoreload
%autoreload 2
%run setup_project.py
prepareSessionsForSpatialAnalysisProject(sSesList,myProject.sessionList)

Project name: autopi_ca1
dataPath: /adata/projects/autopi_ca1
dlcModelPath: /adata/models
Reading /adata/projects/autopi_ca1/sessionList
We have 40 testing sessions in the list
See myProject and sSesList objects
Loading Animal_pose and Spike_train, sSes.ap and sSes.cg


100%|███████████████████████████████████████████| 40/40 [00:38<00:00,  1.05it/s]


Loading ses.trial_table_simple as ses.trials
Create condition intervals in ses.intervalDict


After inspection of several sessions, I find out the transformation required to get the head-direction of the mouse align with its heading.

* 0 : East
* pi/2 : North
* pi : West
* -pi/2: South

The code I used is shown below. Once the correction applied, this code will not work.



The code below can correct for rotation of the HD relative to heading.

In [80]:
from scipy.stats import linregress

In [103]:
overwrite=False

problem_indices = []
for j,ses in enumerate(sSesList):
    print(ses.name,j)
    
    ses.ap.pose_file_extension = ".pose_kf.npy"
    ses.ap.load_pose_from_file()

    ## mean heading 
    delta_x=np.diff(ses.ap.pose[:,1],append=np.nan)
    delta_y=np.diff(ses.ap.pose[:,2],append=np.nan)
    heading = np.arctan2(delta_y,delta_x)
    
    sp = np.sqrt((delta_x**2 + delta_y**2)/(ses.ap.pose[1,0]-ses.ap.pose[0,0]))
    minSpeed=4
    spOk=sp>minSpeed

    h_x = np.cos(heading)
    h_y = np.sin(heading)
    
    hd_x = np.cos(ses.ap.pose[:,4])
    hd_y = np.sin(ses.ap.pose[:,4])
    
    delta_x = h_x-hd_x
    delta_y = h_y-hd_y
    
    
    ## we know that our problem is a 90 degree shift
    ## get the HD shift when heading is 0
    hMin=-np.pi/10
    hMax=np.pi/10
    hSel = np.logical_and(heading>hMin,heading<hMax)
    hSel = np.logical_and(hSel,spOk) # when heading is 0 is True
    hSel = np.logical_and(hSel,~np.isnan(ses.ap.pose[:,4]))
    
      
    res = linregress(heading[hSel],ses.ap.pose[hSel,4])
    
    if (np.abs(res[1]) > np.pi/4):
        
        print(res)
        
        fig, axs = plt.subplots(1,3, figsize=(15,5))

        ax = axs[0]
        ax.scatter(heading[spOk],ses.ap.pose[spOk,4],s=0.01)
        ax.set_xlabel("heading")
        ax.set_ylabel("hd")
        ax.set_title("Original HD")

        ax = axs[1]
        ax.scatter(heading[hSel],ses.ap.pose[hSel,4],s=0.01)
        x = np.arange(-np.pi/10,np.pi/10,0.01)
        ax.plot(x,x*res[0]+res[1],c="red")
        ax.set_xlabel("heading")
        ax.set_ylabel("hd")
              
        # we want to make a correction of 90 or -90
        if res[1] < 0:
            correction = np.pi/2
        else:
            correction = -np.pi/2
        
        # apply the correction
        newHd=ses.ap.pose[:,4]+correction
        newHd = np.arctan2(np.sin(newHd),np.cos(newHd))
        
        if overwrite: # will overwrite the pose data    
            ses.ap.pose_ori[:,4] = newHd
            ax = axs[2]
            ax.scatter(heading[spOk],ses.ap.pose_ori[spOk,4],s=0.01)
            ax.set_xlabel("heading")
            ax.set_ylabel("hd")
            ax.set_title("Corrected HD")
            ses.ap.save_pose_to_file()
        
        else:
            ax = axs[2]
            ax.scatter(heading[spOk],newHd[spOk],s=0.01)
            ax.set_xlabel("heading")
            ax.set_ylabel("hd")
            ax.set_title("Corrected HD")
            
        
        plt.show()

    
    #problem_indices.append(j)


mn5824-20112020-0107 0
mn5824-22112020-0107 1
mn5824-24112020-0107 2
mn5824-02122020-0106 3
mn711-28012021-0106 4
mn711-30012021-0106 5
mn711-31012021-0107 6
mn711-01022021-0107 7
mn711-02022021-0108 8
mn711-03022021-0107 9
mn711-04022021-0107 10
mn2739-11022021-0107 11
mn2739-15022021-0105 12
mn2739-16022021-0106 13
mn2739-17022021-0106 14
mn2739-21022021-0106 15
mn3246-09042021-0106 16
mn3246-10042021-0106 17
mn3246-12042021-0106 18
mn3246-14042021-0106 19
mn1173-02052021-0107 20
mn1173-06052021-0107 21
mn1173-08052021-0107 22
mn1173-09052021-0108 23
mn1173-11052021-0108 24
TYY9524-16082021-0106 25
TYY9524-18082021-0106 26
mn5618-07072021-0107 27
mn5618-09072021-0106 28
mn5618-12072021-0110 29
TYY5622-07092021-0106 30
TYY5622-17092021-0106 31
TYY5622-19092021-0106 32
TYY5622-20092021-0106 33
mn9686-20102021-0106 34
mn9686-26102021-0106 35
mn9686-27102021-0106 36
mn9686-28102021-0107 37
mn9686-29102021-0106 38
mn9686-01112021-0106 39
